In [ ]:
import pandas as pd
from openai import OpenAI

# Initialize Ollama client for Qwen VL
client = OpenAI(
    base_url='http://127.0.0.1:11434/v1',
    api_key='ollama',
)

# Load CoT outputs from first step
cot_csv_path = "/home/intern08/Documents/chest_xray_project/archive/cot_outputs.csv"
df_cot = pd.read_csv(cot_csv_path)

# System prompt for Qwen VL to produce a professional radiology report
system_prompt_report = (
    """
You are a board-certified radiologist.

Consider and analyze the given structured findings inside a <think>...</think> block.

Your task:
1. Write a factual, professional radiology report.
2. ONLY use information explicitly stated in <think>.
3. Do NOT:
   - Guess the patient names, ages, genders, or history if not provided.
   - Add anatomical explanations or textbook definitions.
   - Repeat details excessively.
   - Mention <think>, <cat>, or <area> in the output.
4. Use clear medical language, no casual tone.
5. If <think> is empty, output: "Findings: Not available. Impression: Not available."
"""
)


results_reports = []

for _, row in df_cot.head(5).iterrows():
    user_prompt = (
        f"Chain of Thought:\n{row['cot_output']}\n\n"
        "Generate the radiology report as instructed."
    )

    try:
        response = client.chat.completions.create(
            model="symptoma/medgemma3:27b",
            messages=[
                {"role": "system", "content": [{"type": "text", "text": system_prompt_report}]},
                {"role": "user", "content": [{"type": "text", "text": user_prompt}]}
            ],
            max_tokens=500,
            temperature=0.0,
        )

        report_text = response.choices[0].message.content

        results_reports.append({
            "uid": row["uid"],
            "projection": row["projection"],
            "report": report_text
        })

        # Print the report nicely
        print("="*60)
        print(f"UID: {row['uid']} | Projection: {row['projection']}")
        print("-"*60)
        print(report_text.strip())
        print("="*60 + "\n")

    except Exception as e:
        print(f"Error for UID {row['uid']}: {e}")

# Save reports to CSV
output_reports_path = "/home/intern08/Documents/chest_xray_project/archive/final_reports.csv"
pd.DataFrame(results_reports).to_csv(output_reports_path, index=False)

print(f" All reports saved to {output_reports_path}")
